In [1]:
%reload_ext autoreload
%autoreload 2

### Camera

In [2]:
from datetime import datetime
from pathlib import Path

from inputs import get_gamepad
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
from jetracer.nvidia_racecar import NvidiaRacecar
import ipywidgets
import requests

In [3]:
OUT_P = Path('train_dataset') # directorio de destino de las fotos (se creará)
FPS   = 10 # frames por segundo
SIZE  = 224

In [4]:
moving = False

In [5]:
def update_display():
    info = '\n'.join([
        f"th_gain: {car.throttle_gain:.2f}",
        f"st_offs: {car.steering_offset:.2f}",
    ])
    requests.get("http://localhost:8000/text/" + info)

In [6]:
def reset_display():
    requests.get("http://localhost:8000/stats/on")

In [7]:
def get_timestamp():
    return datetime.utcnow().strftime('%Y%m%d_%H%M%S_%f')

In [20]:
def update_image(change):
    events = get_gamepad()
    image = change['new']
    jpg = bgr8_to_jpeg(image)
    image_widget.value = jpg
    for event in events:
        if event.ev_type == 'Key' and event.state == 1 and event.code == 'BTN_NORTH': # Botó X
            print('xxxxx')
            jpg_p = OUT_P / (get_timestamp() + ".jpg")
            with jpg_p.open('wb') as f:
                f.write(jpg)
            print(f"Th={car.throttle} - saving {jpg_p.name}.jpg")

xxxxx
Th=0.0 - saving 20211222_172600_419153.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_172602_939423.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_172938_740038.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_172944_964191.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_172948_652767.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173000_165353.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173004_733537.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173008_452418.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173012_525194.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173016_131820.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173019_684923.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173022_924769.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173027_389018.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173030_860846.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173033_917571.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173037_429283.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173040_501429.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173044_061543.jpg.jpg
xxxxx
Th=0.0 - saving 20211222_173047_230422.j

In [9]:
car = NvidiaRacecar()

In [10]:
car.steering = 0
car.steering_offset = 0
car.throttle_gain = 0.20

In [11]:
OUT_P.mkdir(exist_ok=True)

In [12]:
camera = CSICamera(width=SIZE, height=SIZE, capture_fps=FPS)
camera.running = True

In [13]:
image_widget = ipywidgets.Image(format='jpeg')

In [14]:
camera.observe(update_image, names='value')

In [15]:
display(image_widget)

Image(value=b'', format='jpeg')

In [19]:
# EXECUTAR NOMÉS QUAN VOLEM FOTOS EN MOVIMENT. NO EXECUTAR QUAN FEM FOTOS A LA X
right_trigger = left_trigger = 0

while not (right_trigger and left_trigger):
    events = get_gamepad()
    for event in events:
        if event.ev_type == 'Absolute':
            if event.code == 'ABS_Y':
                car.throttle = -(event.state - 127.5) / 127.5
                #print(f'Y={event.state}')
            if event.code == 'ABS_Z':
                car.steering = (event.state - 127.5) / 127.5
                #print(f'Z={event.state}')
        elif event.ev_type == 'Key' and event.code == 'BTN_TR2':
            right_trigger = event.state
        elif event.ev_type == 'Key' and event.code == 'BTN_TL2':
            left_trigger = event.state
        elif event.ev_type == 'Key' and event.state == 1:
            if event.code == 'BTN_WEST': # Y / UP
                car.throttle_gain = min(1.0, car.throttle_gain + 0.05)
                update_display()
            if event.code == 'BTN_SOUTH': # A / DOWN
                car.throttle_gain = max(0.0, car.throttle_gain - 0.05)
                update_display()
            if event.code == 'BTN_EAST': # B / RIGHT
                car.steering_offset = max(-0.3, car.steering_offset - 0.05)
                car.steering = 0.1
                car.steering = 0
                update_display()
            if event.code == 'BTN_NORTH': # X / LEFT
                #car.steering_offset = min(0.3, car.steering_offset + 0.05)
                #car.steering = 0.1
                #car.steering = 0
                update_display()

            #print(f"throttle_gain={car.throttle_gain:.1f}")
        #print(event.ev_type, event.code, event.state)

car.throttle = 0
reset_display()

In [17]:
#camera.unobserve(update_image, names='value')

In [18]:
#camera.running = False